# MariaDB
## Istallation
The following command installs both, the server and the client:
```
sudo apt install mariadb-server
```

## Create User Account
```
GRANT ALL ON compbiol.* TO awkologist@localhost IDENTIFIED BY "awkology";
```

## Create Tables
```
CREATE TABLE annotation (gene VARCHAR(7), function VARCHAR(30), metabolism VARCHAR(30));
CREATE TABLE expression(gene VARCHAR(7), expr_value INT);
```

## Download Data Files

In [2]:
wget 'https://github.com/awkologist/CompBiol3/raw/main/MariaDB/annotation.tab'

--2024-03-04 16:46:34--  https://github.com/awkologist/CompBiol3/raw/main/MariaDB/annotation.tab
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/awkologist/CompBiol3/main/MariaDB/annotation.tab [following]
--2024-03-04 16:46:35--  https://raw.githubusercontent.com/awkologist/CompBiol3/main/MariaDB/annotation.tab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 368 [text/plain]
Saving to: ‘annotation.tab.1’

annotation.tab.1    100%[===================>]     368  --.-KB/s    in 0s      

2024-03-04 16:46:35 (1.16 MB/s) - ‘annotation.tab.1’ saved [368/368]



In [ ]:
wget 'https://github.com/awkologist/CompBiol3/raw/main/MariaDB/expression.tab'

## Populate Tables
```
mariadb-import --delete --ignore-lines=1 compbiol annotation.tab
mariadb-import --delete --ignore-lines=1 compbiol expression.tab
```

## Querying

In [4]:
mariadb -e 'SELECT * FROM expression LIMIT 3;'

+---------+------------+
| gene    | expr_value |
+---------+------------+
| alr1207 |       8303 |
| alr2938 |      10323 |
| alr3395 |       1432 |
+---------+------------+


In [5]:
mariadb -e 'SELECT * FROM expression WHERE expr_value < 1000;'

+---------+------------+
| gene    | expr_value |
+---------+------------+
| alr4392 |        729 |
| alr4851 |        633 |
+---------+------------+


In [6]:
mariadb -e 'SELECT * FROM expression WHERE expr_value < 1000  ORDER BY expr_value;'

+---------+------------+
| gene    | expr_value |
+---------+------------+
| alr4851 |        633 |
| alr4392 |        729 |
+---------+------------+


In [8]:
mariadb -e 'SELECT * FROM annotation WHERE function LIKE "%ase%";'

+---------+--------------------------------+-------------------------------+
| gene    | function                       | metabolism                    |
+---------+--------------------------------+-------------------------------+
| alr2938 | iron superoxide dismutase      | Detoxification                |
| alr4851 | preprotein translocase subunit | Protein and peptide secretion |
| alr3395 | adenylosuccinate lyase         | Purine biosynthesis           |
| alr1207 | uridylate kinase               | Pyrimidine biosynthesis       |
| alr5000 | CTP synthetase                 | Pyrimidine biosynthesis       |
| all3556 | succinate-dehydrogenase        | TCA cycle                     |
+---------+--------------------------------+-------------------------------+


In [9]:
mariadb -e 'SELECT * FROM annotation WHERE metabolism REGEXP "(Purine|Pyrimidine) biosynthesis";'

+---------+------------------------+-------------------------+
| gene    | function               | metabolism              |
+---------+------------------------+-------------------------+
| alr3395 | adenylosuccinate lyase | Purine biosynthesis     |
| alr1207 | uridylate kinase       | Pyrimidine biosynthesis |
| alr5000 | CTP synthetase         | Pyrimidine biosynthesis |
+---------+------------------------+-------------------------+


In [12]:
mariadb -e 'SELECT gene, function FROM annotation WHERE metabolism REGEXP "(Purine|Pyrimidine) biosynthesis"\G'

*************************** 1. row ***************************
    gene: alr3395
function: adenylosuccinate lyase
*************************** 2. row ***************************
    gene: alr1207
function: uridylate kinase
*************************** 3. row ***************************
    gene: alr5000
function: CTP synthetase


In [13]:
mariadb -e 'SELECT MIN(expr_value) FROM expression;'

+-----------------+
| MIN(expr_value) |
+-----------------+
|             633 |
+-----------------+


In [15]:
mariadb -e 'SELECT annotation.gene, annotation.function, expression.expr_value FROM annotation, expression WHERE annotation.gene = expression.gene AND annotation.metabolism REGEXP "(Purine|Pyrimidine) biosynthesis" AND expression.expr_value < 5000;'

+---------+------------------------+------------+
| gene    | function               | expr_value |
+---------+------------------------+------------+
| alr3395 | adenylosuccinate lyase |       1432 |
+---------+------------------------+------------+


In [16]:
mariadb -e 'SELECT a.gene, a.function, e.expr_value FROM annotation AS a, expression AS e WHERE a.gene = e.gene AND a.metabolism REGEXP "(Purine|Pyrimidine) biosynthesis" AND e.expr_value < 5000;'

+---------+------------------------+------------+
| gene    | function               | expr_value |
+---------+------------------------+------------+
| alr3395 | adenylosuccinate lyase |       1432 |
+---------+------------------------+------------+


In [17]:
mariadb -e 'SELECT a.gene, a.function, e.expr_value FROM annotation AS a LEFT JOIN expression AS e ON a.gene = e.gene WHERE a.metabolism REGEXP "(Purine|Pyrimidine) biosynthesis" AND e.expr_value < 5000;'

+---------+------------------------+------------+
| gene    | function               | expr_value |
+---------+------------------------+------------+
| alr3395 | adenylosuccinate lyase |       1432 |
+---------+------------------------+------------+


Create the following file:
```
# save as query.sql
USE compbiol
SELECT a.function, e.expr_value 
FROM annotation AS a LEFT JOIN expression AS e ON a.gene = e.gene 
WHERE e.expr_value < 5000;

In [18]:
mariadb < query.sql

function	expr_value
nitrogen-responsive regulator	729
preprotein translocase subunit	633
adenylosuccinate lyase	1432


In [19]:
mariadb < query.sql | sed '1d' | awk -F"\t" '{print $1}'

nitrogen-responsive regulator
preprotein translocase subunit
adenylosuccinate lyase
